## PASO 1. Comprender el objetivo

En este cuaderno de Colaboratory, voy a desarrollar un sistema de predicción (o, mejor dicho para este caso, estimación) de enfermedades cardiovasculares en diferentes personas.

Partiremos del conjunto de datos almacenado en el archivo 'cardio_train.csv', el cual contiene las siguientes columnas:
  - Age
  - Height
  - Weight
  - Gender
  - Smoking
  - Alcohol intake
  - Physical activity
  - Systolic blood pressure
  - Diastolic blood pressure
  - Cholesterol
  - Glucose

Nuestro modelo deberá ser capaz de, basandonos en los valores que contengan estos atributos en un determinado registro, determinar si esa persona **podría estar sufriendo alguna enfermedad cardiovascular**.


## PASO 2. Importar librerías y archivos necesarios

In [1]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf

import matplotlib.pyplot as plt

In [2]:
registro_medico_df = pd.read_csv('cardio_train.csv')

## PASO 3. Analizar los datos que tenemos

In [3]:
registro_medico_df.head()

,id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio
0,0;18393;2;168;62.0;110;80;1;1;0;0;1;0
1,1;20228;1;156;85.0;140;90;3;1;0;0;1;1
2,2;18857;1;165;64.0;130;70;3;1;0;0;0;1
3,3;17623;2;169;82.0;150;100;1;1;0;0;1;1
4,4;17474;1;156;56.0;100;60;1;1;0;0;0;0


In [4]:
registro_medico_df.tail()

,id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio
69995,99993;19240;2;168;76.0;120;80;1;1;1;0;1;0
69996,99995;22601;1;158;126.0;140;90;2;2;0;0;1;1
69997,99996;19066;2;183;105.0;180;90;3;1;0;1;0;1
69998,99998;22431;1;163;72.0;135;80;1;2;0;0;0;1
69999,99999;20540;1;170;72.0;120;80;2;1;0;0;1;0


In [5]:
registro_medico_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 1 columns):
 #   Column                                                                             Non-Null Count  Dtype 
---  ------                                                                             --------------  ----- 
 0   id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio  70000 non-null  object
dtypes: object(1)
memory usage: 547.0+ KB


In [6]:
registro_medico_df.columns

Index(['id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio'], dtype='object')

In [7]:
# Tras la ejecución de las celdas anteriores, veo como realmente, mi dataframe ha registrado toda la información en una sola columna.
# Voy a hacer lo siguiente para corregir este error y empezar a trabajar con los datos:
#   1. Creo las nuevas columnas del dataframe.
#   2. Asigno los valores de cada columna (están separados por el delimitador ';', almacenados en una sola columna.)
#   3. Inserto cada lista en su columna correspondiente.

In [8]:
registro_medico_df['Valores'] = registro_medico_df['id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio'].apply(lambda x: x.split(';'))
print(registro_medico_df)

      id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio  \
0                  0;18393;2;168;62.0;110;80;1;1;0;0;1;0                                  
1                  1;20228;1;156;85.0;140;90;3;1;0;0;1;1                                  
2                  2;18857;1;165;64.0;130;70;3;1;0;0;0;1                                  
3                 3;17623;2;169;82.0;150;100;1;1;0;0;1;1                                  
4                  4;17474;1;156;56.0;100;60;1;1;0;0;0;0                                  
...                                                  ...                                  
69995          99993;19240;2;168;76.0;120;80;1;1;1;0;1;0                                  
69996         99995;22601;1;158;126.0;140;90;2;2;0;0;1;1                                  
69997         99996;19066;2;183;105.0;180;90;3;1;0;1;0;1                                  
69998          99998;22431;1;163;72.0;135;80;1;2;0;0;0;1                                  

In [9]:
registro_medico_df[['ID',
                    'Age',
                    'Gender',
                    'Height',
                    'Weight',
                    'ap_hi',
                    'ap_lo',
                    'Cholesterol',
                    'Gluc',
                    'Smoke',
                    'Alco',
                    'Active',
                    'Cardio']] = pd.DataFrame(registro_medico_df['Valores'].tolist(), index=registro_medico_df.index)
print(registro_medico_df)

      id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio  \
0                  0;18393;2;168;62.0;110;80;1;1;0;0;1;0                                  
1                  1;20228;1;156;85.0;140;90;3;1;0;0;1;1                                  
2                  2;18857;1;165;64.0;130;70;3;1;0;0;0;1                                  
3                 3;17623;2;169;82.0;150;100;1;1;0;0;1;1                                  
4                  4;17474;1;156;56.0;100;60;1;1;0;0;0;0                                  
...                                                  ...                                  
69995          99993;19240;2;168;76.0;120;80;1;1;1;0;1;0                                  
69996         99995;22601;1;158;126.0;140;90;2;2;0;0;1;1                                  
69997         99996;19066;2;183;105.0;180;90;3;1;0;1;0;1                                  
69998          99998;22431;1;163;72.0;135;80;1;2;0;0;0;1                                  

In [10]:
registro_medico_df = registro_medico_df.drop(columns = ['id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio','Valores'])

In [11]:
registro_medico_df

,ID,Age,Gender,Height,Weight,ap_hi,ap_lo,Cholesterol,Gluc,Smoke,Alco,Active,Cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [12]:
# Ya tengo el dataframe listo para continuar con el análisis de los datos

registro_medico_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           70000 non-null  object
 1   Age          70000 non-null  object
 2   Gender       70000 non-null  object
 3   Height       70000 non-null  object
 4   Weight       70000 non-null  object
 5   ap_hi        70000 non-null  object
 6   ap_lo        70000 non-null  object
 7   Cholesterol  70000 non-null  object
 8   Gluc         70000 non-null  object
 9   Smoke        70000 non-null  object
 10  Alco         70000 non-null  object
 11  Active       70000 non-null  object
 12  Cardio       70000 non-null  object
dtypes: object(13)
memory usage: 6.9+ MB


In [13]:
registro_medico_df.describe()

,ID,Age,Gender,Height,Weight,ap_hi,ap_lo,Cholesterol,Gluc,Smoke,Alco,Active,Cardio
count,70000,70000,70000,70000,70000,70000,70000,70000,70000,70000,70000,70000,70000
unique,70000,8076,2,109,287,153,157,3,3,2,2,2,2
top,0,19741,1,165,65.0,120,80,1,1,0,0,1,0
freq,1,32,45530,5853,3850,27699,34847,52385,59479,63831,66236,56261,35021


In [14]:
# La columna 'Age' almacena los valores registrados en un formato de días, el cual no es muy conveniente.
# Voy a convertir esots valores a años, y además los voy a redondear al alza haciendo uso del metodo .round() de Python

# registro_medico_df['Age'] = registro_medico_df['Age'] / 365

# AVISO: al ejecutar la línea comentada anterior a esta, la celda me manda el siguiente error:
# TypeError: unsupported operand type(s) for /: 'str' and 'int'
# Parece ser que la columna almacena los valores en formato string, por lo que va a ser necesario cambiar el tipo de dato antes de pasar los días a años.

In [15]:
registro_medico_df['Age'] = registro_medico_df['Age'].astype(int)

In [16]:
registro_medico_df['Age'] = registro_medico_df['Age'] / 365

In [17]:
registro_medico_df.head()

,ID,Age,Gender,Height,Weight,ap_hi,ap_lo,Cholesterol,Gluc,Smoke,Alco,Active,Cardio
0,0,50.391781,2,168,62.0,110,80,1,1,0,0,1,0
1,1,55.419178,1,156,85.0,140,90,3,1,0,0,1,1
2,2,51.663014,1,165,64.0,130,70,3,1,0,0,0,1
3,3,48.282192,2,169,82.0,150,100,1,1,0,0,1,1
4,4,47.873973,1,156,56.0,100,60,1,1,0,0,0,0


In [18]:
registro_medico_df['Age'] = round(registro_medico_df['Age'],0)

In [19]:
registro_medico_df.head()

,ID,Age,Gender,Height,Weight,ap_hi,ap_lo,Cholesterol,Gluc,Smoke,Alco,Active,Cardio
0,0,50.0,2,168,62.0,110,80,1,1,0,0,1,0
1,1,55.0,1,156,85.0,140,90,3,1,0,0,1,1
2,2,52.0,1,165,64.0,130,70,3,1,0,0,0,1
3,3,48.0,2,169,82.0,150,100,1,1,0,0,1,1
4,4,48.0,1,156,56.0,100,60,1,1,0,0,0,0


In [20]:
registro_medico_df['ID'] = registro_medico_df['ID'].astype(int)
registro_medico_df['Gender'] = registro_medico_df['Gender'].astype(int)
registro_medico_df['Height'] = registro_medico_df['Height'].astype(int)
registro_medico_df['Weight'] = registro_medico_df['Weight'].astype(float)
registro_medico_df['ap_hi'] = registro_medico_df['ap_hi'].astype(int)
registro_medico_df['ap_lo'] = registro_medico_df['ap_lo'].astype(int)
registro_medico_df['Cholesterol'] = registro_medico_df['Cholesterol'].astype(int)
registro_medico_df['Gluc'] = registro_medico_df['Gluc'].astype(int)
registro_medico_df['Smoke'] = registro_medico_df['Smoke'].astype(int)
registro_medico_df['Alco'] = registro_medico_df['Alco'].astype(int)
registro_medico_df['Active'] = registro_medico_df['Active'].astype(int)
registro_medico_df['Cardio'] = registro_medico_df['Cardio'].astype(int)

In [21]:
registro_medico_df.dtypes

ID               int64
Age            float64
Gender           int64
Height           int64
Weight         float64
ap_hi            int64
ap_lo            int64
Cholesterol      int64
Gluc             int64
Smoke            int64
Alco             int64
Active           int64
Cardio           int64
dtype: object

In [22]:
# Ahora los datos son de tipo numérico, puedo usar el metodo pd.describe() para tener una mejor representación del conjunto de datos.
registro_medico_df.describe()

,ID,Age,Gender,Height,Weight,ap_hi,ap_lo,Cholesterol,Gluc,Smoke,Alco,Active,Cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,53.338686,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,6.765294,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,30.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,48.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,54.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,58.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,65.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
registro_medico_df.isnull().sum()

ID             0
Age            0
Gender         0
Height         0
Weight         0
ap_hi          0
ap_lo          0
Cholesterol    0
Gluc           0
Smoke          0
Alco           0
Active         0
Cardio         0
dtype: int64

In [24]:
# El dataframe no contiene valores nulos, pero si datso un poco extraños. En el dataframe resultante tras la ejecución del método pd.describe(), se muestra
# como aparecen algunos valores un poco extraños, como:
#   - El valor mínimo registrado para un persona, es de 10 kilos. No creo que en el registro de los datos se contemple la existencia de pacientes bebes,
#     por lo que voy a asumir que es un error. Ahora lo corregiré.
#   - Los valores mínimos para las columnas 'ap_hi', 'ap_lo' son negativos, por lo que tendré que eliminar dichas filas.
#   - Por otro lado , las columnas 'ap_hi', 'ap_lo' registran también valores anormales como valores máximos, por lo que también debo de eliminarlos.

In [25]:
# Con esta línea, elimino aquellas filas donde se cumpla que la columna 'Weight' almacene un valor inferior a 30.0 kilogramos.
# Pongo 30 kg como condicion y no un valor más alto debido a que es posible que algunos adultos puedan tener trastornos alimenticios,
# y estaría bien que el sistema contemplase también esta posibilidad.

# Seleccionamos las filas que cumplen la condición
peso_erroneo = registro_medico_df.loc[registro_medico_df['Weight'] < 30.0]

# Eliminamos las filas seleccionadas
registro_medico_df = registro_medico_df.drop(peso_erroneo.index)


In [26]:
# Estas dos líneas van a eliminar aquellas filas del dataframe que contienen un valor negativo para las columnas 'ap_hi', 'ap_lo'.
p_sistolica_baja = registro_medico_df.loc[registro_medico_df['ap_hi'] < 70.0] # Asigno 70 como valor mínimo para este tipo de presión
registro_medico_df = registro_medico_df.drop(p_sistolica_baja.index)

p_diastolica_baja = registro_medico_df.loc[registro_medico_df['ap_lo'] < 40.0] # Asigno 40 como valor mínimo para este tipo de presión
registro_medico_df = registro_medico_df.drop(p_diastolica_baja.index)

#registro_medico_df = registro_medico_df.loc[registro_medico_df['ap_hi'] < 0]
#registro_medico_df = registro_medico_df.loc[registro_medico_df['ap_lo'] < 0]

In [27]:
# Por último , voy a eliminar los valores para las columnas 'ap_hi', 'ap_lo' que superan un rango crítico para la salud.
p_sistolica_alta = registro_medico_df.loc[registro_medico_df['ap_hi'] > 210.0]
registro_medico_df = registro_medico_df.drop(p_sistolica_alta.index)

p_diastolica_alta = registro_medico_df.loc[registro_medico_df['ap_lo'] > 140.0]
registro_medico_df = registro_medico_df.drop(p_diastolica_alta.index)

#registro_medico_df = registro_medico_df.loc[registro_medico_df['ap_hi'] > 210]
#registro_medico_df = registro_medico_df.loc[registro_medico_df['ap_lo'] > 140]

# Cada una de las columnas representa un valor asignado:
#   - 'ap_hi' representa la presión sistolica, cuyo valor crítico comienza a partir de 180 (asigno 210 para dejar algo de margen).
#   - 'ap_lo' representa la presión diastolica, cuyo valor crítico comienza a partir de 110 (asigno 140 para dejar algo de margen).

In [28]:
registro_medico_df.describe()

,ID,Age,Gender,Height,Weight,ap_hi,ap_lo,Cholesterol,Gluc,Smoke,Alco,Active,Cardio
count,68711.000000,68711.000000,68711.000000,68711.000000,68711.000000,68711.000000,68711.000000,68711.000000,68711.000000,68711.000000,68711.000000,68711.000000,68711.000000
mean,49969.475441,53.325013,1.348678,164.360961,74.118853,126.574129,81.333731,1.364425,1.225801,0.088035,0.053543,0.803379,0.494637
std,28845.678062,6.768030,0.476555,8.183268,14.317451,16.620014,9.497952,0.678793,0.571757,0.283348,0.225115,0.397446,0.499975
min,0.000000,30.000000,1.000000,55.000000,30.000000,70.000000,40.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,24993.500000,48.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50007.000000,54.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74864.500000,58.000000,2.000000,170.000000,82.000000,140.000000,90.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,65.000000,2.000000,250.000000,200.000000,210.000000,140.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


## PASO 4. Visualizar el conjunto de datos

In [29]:
# ...

## PASO 5. Dividir el conjunto de entrenamiento

In [58]:
X = registro_medico_df.drop(columns = ['Cardio'])
y = registro_medico_df['Cardio']

In [31]:
X.shape

(68711, 12)

In [32]:
y.shape

(68711,)

In [33]:
#X = np.array(X).astype('float32')
#y = np.array(y).astype('float32')

In [34]:
#y = np.reshape(y, (-1,1))

In [35]:
#y.shape

(68711, 1)

In [59]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 0)

In [37]:
X_train.shape

(54968, 12)

In [38]:
X_test.shape

(13743, 12)

In [39]:
y_train.shape

(54968, 1)

In [40]:
y_test.shape

(13743, 1)

## PASO 6. Crear el modelo y entrenarlo

In [52]:
# Importar la clase de modelo que deseamos utilizar
from sklearn.tree import DecisionTreeClassifier

# Crear una instancia del modelo
model = DecisionTreeClassifier()

In [53]:
# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [54]:
# Realizar predicciones sobre los datos de prueba
y_pred = model.predict(X_test)


# Evaluar el rendimiento del modelo
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.6294113366804919


In [55]:
# Muestro las diez primeras filas del conjunto que uso para que el modelo haga predicciones

X_test[:10]

,ID,Age,Gender,Height,Weight,ap_hi,ap_lo,Cholesterol,Gluc,Smoke,Alco,Active
42524,60752,54.0,1,168,64.0,120,70,1,3,0,0,0
6457,9182,44.0,1,162,128.0,140,90,1,1,0,0,1
51775,73827,60.0,2,175,90.0,120,80,3,3,0,0,1
20949,29913,56.0,1,158,71.0,160,80,1,1,1,0,1
22770,32523,62.0,1,160,61.0,110,80,1,1,0,0,0
26150,37346,41.0,1,165,63.0,110,70,1,1,0,0,1
32126,45880,64.0,2,173,69.0,120,80,1,1,1,0,1
36842,52654,60.0,1,163,71.0,100,60,1,1,0,0,1
32599,46551,50.0,1,165,72.0,120,80,1,1,0,0,1
31905,45564,44.0,1,169,66.0,150,90,2,1,0,0,1


In [57]:
# Muestro las predicciones para cada uno de los 10 valores mostrados con la ejecución de la celda anterior.
# Un valor de 1, representa que la modelo ha predicho que la persona tiene un problema cardíaco.
# Un valor de 0, representa que la modelo ha predicho que la persona no tiene ningún problema cardíaco.
y_pred[:10]

array([0, 1, 1, 1, 1, 0, 0, 0, 0, 0])